In [1]:
import torch
from torch.utils.data import TensorDataset,DataLoader,Dataset
from torch import nn

import logomaker
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
random.seed(7) #10

import utils as u

In [2]:
def load_data(
        upstream_region_file,
        data_mat_file, 
        sample2cond_file, 
        sample_file, 
        condition_file,
        coded_meta_file
        ):
    '''
    Wrapper function to load data from files into relavent objects
    '''
    # load upstream seq regions
    seqs = u.load_promoter_seqs(upstream_region_file)
    loc2seq = dict([(x,z) for (x,y,z) in seqs])
    
    # load TPM data
    tpm_df = pd.read_csv(data_mat_file,sep='\t').fillna('')

    
    # load mapping from sample to condition
    with open(sample2cond_file,'r') as f:
        sample2condition = dict(x.strip().split() for x in f.readlines())

    
    # load sample to include file
    if sample_file:
        with open(sample_file,'r') as f:
            samples = list(x.strip() for x in f.readlines())
    # if none provided, just use all the samples from the sample2condition dict
    else: 
        samples = list(sample2condition.keys())

        
    # load the conditions to include file
    if condition_file:
        with open(condition_file,'r') as f:
            conditions = list(x.strip() for x in f.readlines())
    # if none provided, just use all the conditions
    else:
        conditions = list(set([sample2condition[x] for x in sample2condition]))

    # load coded metadata file
    meta_df = pd.read_csv(coded_meta_file,sep='\t')
    meta_df['sample'] = meta_df['#sample']+'_tpm'

    return loc2seq, tpm_df, sample2condition, samples, conditions, meta_df

In [3]:
upstream_region_file = 'all_seq_info/all_loci_upstream_regions_w100_min20.fa'
data_mat_file = 'data/extract_TPM_counts.tsv'
sample2cond_file = 'data/sample2condition.txt'
sample_file = None
condition_file = 'data/conditions_to_include.txt'
coded_meta_file = 'data/5G_exp_metadata_coded.tsv'
COND_COLS = ['carbon_source','oxygen_level','nitrate_level','copper_level','lanthanum_level','growth_rate','growth_mode']


loc2seq, tpm_df, sample2condition, samples, conditions, meta_df = load_data(
    upstream_region_file,
    data_mat_file, 
    sample2cond_file, 
    sample_file, 
    condition_file,
    coded_meta_file
)

In [4]:
tpm_df.head()

,locus_tag,product,type,gene_symbol,locus,start_coord,end_coord,note,translation,gene_len,...,5GB1_pA9_red_tpm,5GB1_pA9_yellow_tpm,5GB1C-5G-La-BR1_tpm,5GB1C-5G-La-BR2_tpm,5GB1C-5G-N-BR1_tpm,5GB1C-5G-N-BR2_tpm,5GB1C-JG15-La-BR1_tpm,5GB1C-JG15-La-BR2_tpm,5GB1C-JG15-N-BR1_tpm,5GB1C-JG15-N-BR2_tpm
0,EQU24_RS00005,chromosomal replication initiator protein DnaA,CDS,dnaA,NZ_CP035467.1,0,1317,Derived by automated computational analysis us...,MSALWNNCLAKLENEISSSEFSTWIRPLQAIETDGQIKLLAPNRFV...,1318,...,38.557373,38.810668,37.444214,40.246006,40.100118,33.432274,39.880174,38.355431,30.247582,41.248441
1,EQU24_RS00010,DNA polymerase III subunit beta,CDS,,NZ_CP035467.1,1502,2603,Derived by automated computational analysis us...,MKYIINREQLLVPLQQIVSVIEKRQTMPILSNVLMVFRENTLVMTG...,1102,...,52.552767,52.461746,42.676553,49.210083,46.798476,48.142385,45.465136,46.498139,37.152951,52.902410
2,EQU24_RS00015,DNA replication/repair protein RecF,CDS,recF,NZ_CP035467.1,3060,4140,Derived by automated computational analysis us...,MSLQKLDIFNVRNIRQASLQPSPGLNLIYGANASGKSSVLEAIFIL...,1081,...,31.350991,34.914128,21.479309,24.204682,22.171104,22.006566,22.658157,22.753325,19.407103,29.834124
3,EQU24_RS00020,DNA topoisomerase (ATP-hydrolyzing) subunit B,CDS,gyrB,NZ_CP035467.1,4185,6600,Derived by automated computational analysis us...,MSENIKQYDSTNIQVLKGLDAVRKRPGMYIGDTDDGTGLHHMVFEV...,2416,...,74.848501,80.850761,54.959319,64.911376,59.653059,64.648318,69.119079,65.643179,57.590223,68.306759
4,EQU24_RS00025,hypothetical protein,CDS,,NZ_CP035467.1,6825,7062,Derived by automated computational analysis us...,VKTTKYFLTTRMRPDREIIKDEWIQYVVRFPENEHIQFDGRIRRWA...,238,...,50.324948,49.349547,34.539657,36.521074,37.789611,39.358066,38.992158,35.870964,41.462392,40.227192


In [5]:
locus2info = u.make_info_dict(tpm_df)
locus2info['EQU24_RS19315']

{'gene': 'pmoC',
 'product': 'methane monooxygenase/ammonia monooxygenase subunit C',
 'type': 'CDS'}

In [6]:
df_means = u.get_gene_means_by_condition(tpm_df,samples,sample2condition)

In [7]:
df_means

locus_tag,exp_condition,EQU24_RS00005,EQU24_RS00010,EQU24_RS00015,EQU24_RS00020,EQU24_RS00025,EQU24_RS00030,EQU24_RS00035,EQU24_RS00040,EQU24_RS00045,...,EQU24_RS22110,EQU24_RS22115,EQU24_RS22120,EQU24_RS22125,EQU24_RS22130,EQU24_RS22135,EQU24_RS22140,EQU24_RS22145,EQU24_RS22150,EQU24_RS22155
0,LanzaTech,25.626702,55.711920,11.804042,76.880335,43.737438,27.940983,35.296053,25.926725,23.461781,...,7420.641716,16.014544,15.947067,16.286482,9.514666,46.013885,354.315359,157.364073,571.458102,613.084675
1,MeOH,23.323155,18.905775,18.443916,18.257805,16.950643,12.367795,43.805536,9.660950,7.292145,...,1298.247682,15.614619,20.198066,25.994364,20.950234,28.709983,93.606437,161.518124,496.980651,280.334047
2,NO3_lowO2_slow_growth,32.040358,43.646760,21.341623,62.257687,41.674925,31.911455,57.839768,16.875694,14.916147,...,6497.858109,26.263485,28.935133,23.515245,26.422667,35.157264,178.986199,164.073806,433.428735,493.885115
3,NoCu,44.338687,59.619360,28.258717,56.808319,49.829406,38.384652,81.520362,40.491969,36.566500,...,8345.775345,43.055124,34.370565,44.409579,34.591933,65.329879,253.598495,273.274694,731.042190,1087.611126
4,NoLanthanum,33.434023,43.679839,23.162675,57.287047,42.357072,41.931657,102.503601,30.216787,19.452312,...,5085.627409,16.413284,35.578138,44.613117,43.191743,21.917260,109.773330,67.267718,211.565175,328.933746
5,WT_control,34.988452,38.272163,15.495930,54.855025,35.396786,26.817623,59.558466,24.797995,27.925137,...,4939.367129,18.248719,21.027514,31.490596,45.810743,36.195898,172.298263,181.468991,403.551659,648.122601
6,WithLanthanum,35.452185,41.782237,20.634554,57.120166,34.248335,46.191637,110.711781,31.803805,19.428086,...,3942.947792,15.962203,34.308829,49.206725,39.990662,21.210809,98.090610,73.106973,194.379586,319.988959
7,aa3_KO,26.849583,45.489035,11.962099,55.846659,33.097353,28.718155,50.249664,21.490715,24.790508,...,5355.094603,16.236806,20.836285,28.298842,40.627394,30.065630,181.790564,153.928317,376.664997,657.228922
8,crotonic_acid,38.684021,52.507256,33.132559,77.849631,49.837247,37.332021,69.675355,32.001867,35.856260,...,8680.904376,42.175374,47.879804,58.961083,43.069352,81.481463,365.899345,315.553709,614.920997,676.638496
9,highCu,47.851477,79.099490,33.524043,73.320408,48.652214,33.976359,92.989818,51.940784,50.360579,...,8132.537467,48.884308,35.598730,46.099300,30.115207,89.700150,342.971435,386.483127,1021.443762,1692.391154


In [8]:
# filter to only the copper conditions
cu_conds = ['NoCu','lowCu','medCu','highCu']

df_means_cu = df_means[df_means['exp_condition'].isin(cu_conds)]
df_means_cu

locus_tag,exp_condition,EQU24_RS00005,EQU24_RS00010,EQU24_RS00015,EQU24_RS00020,EQU24_RS00025,EQU24_RS00030,EQU24_RS00035,EQU24_RS00040,EQU24_RS00045,...,EQU24_RS22110,EQU24_RS22115,EQU24_RS22120,EQU24_RS22125,EQU24_RS22130,EQU24_RS22135,EQU24_RS22140,EQU24_RS22145,EQU24_RS22150,EQU24_RS22155
3,NoCu,44.338687,59.619360,28.258717,56.808319,49.829406,38.384652,81.520362,40.491969,36.566500,...,8345.775345,43.055124,34.370565,44.409579,34.591933,65.329879,253.598495,273.274694,731.042190,1087.611126
9,highCu,47.851477,79.099490,33.524043,73.320408,48.652214,33.976359,92.989818,51.940784,50.360579,...,8132.537467,48.884308,35.598730,46.099300,30.115207,89.700150,342.971435,386.483127,1021.443762,1692.391154
12,lowCu,42.963556,61.199155,28.818713,61.563321,50.956799,31.309574,75.037593,40.028670,35.321019,...,7157.334557,43.376082,33.564108,36.862718,31.889782,66.733497,293.589291,313.731841,843.597251,1123.659681
15,medCu,44.910897,65.001074,29.409165,65.383162,48.719958,26.307981,80.114744,39.235705,32.118273,...,5934.158113,43.944096,36.386184,34.983692,32.897138,84.616668,340.021564,377.209038,1060.040784,1429.583942


In [9]:
# transpose to get genes as examples again and add upstream region
XYdf = df_means_cu.set_index('exp_condition').T.reset_index()
XYdf['upstream_region'] = XYdf['locus_tag'].apply(lambda x: loc2seq[x])
XYdf = XYdf[['locus_tag','upstream_region']+cu_conds]

XYdf

exp_condition,locus_tag,upstream_region,NoCu,lowCu,medCu,highCu
0,EQU24_RS00005,CGCCGGTTTATGTCAATTATGCCGGCACTGATTTGATTGCTGTATA...,44.338687,42.963556,44.910897,47.851477
1,EQU24_RS00010,AACGCCGGTTTTACAGTTCATAAGCTATTGATAAATAAAATAAAAA...,59.619360,61.199155,65.001074,79.099490
2,EQU24_RS00015,ATCGCAGTCATTATTAAATGTGGAAGCAACAAAAAAACGAGCTTGT...,28.258717,28.818713,29.409165,33.524043
3,EQU24_RS00020,AACTTAATAACTATAAAATGTTCCACGTGGAACATGGTGAAATTAA...,56.808319,61.563321,65.383162,73.320408
4,EQU24_RS00025,CTTTGCCGAACACCCCGCACCTCCACGCGTCAACAACGAAATTTGA...,49.829406,50.956799,48.719958,48.652214
...,...,...,...,...,...,...
4208,EQU24_RS22135,CCCGGCCGGTTTGGTCTTGTACTGGGTGGTCAACAATACGCTGTCG...,65.329879,66.733497,84.616668,89.700150
4209,EQU24_RS22140,GCCGCCCAGGGCACCTATCTTACAGTCCGAAGAGTATTAAAGTGTC...,253.598495,293.589291,340.021564,342.971435
4210,EQU24_RS22145,AATATTGATGTTGTTGTTATGGCCCGAAAAGATGCACTCAATGCAT...,273.274694,313.731841,377.209038,386.483127
4211,EQU24_RS22150,AAGAACTCACGGCTTTCGTGCCAGAATGGCGACCAAAGGCGGCCGT...,731.042190,843.597251,1060.040784,1021.443762


In [10]:
loc2seq["EQU24_RS19315"]

'GCAGCACCAAATTGGACTGGTAGAGCTTAAATAAAAGCGTTAAAGGGATGTTTTAAAACAACCGCCCTTCGGGGTTTTTAAAAATTTTTTAGGAGGTAGA'

In [11]:
def one_hot_encode(seq):
    #print("one hot encoding...")
    
    # Dictionary returning one-hot encoding of nucleotides. 
    nuc_d = {'A':[1.0,0.0,0.0,0.0],
             'C':[0.0,1.0,0.0,0.0],
             'G':[0.0,0.0,1.0,0.0],
             'T':[0.0,0.0,0.0,1.0],
             'N':[0.0,0.0,0.0,0.0]}
    
    # Creat empty matrix.
    #vec=torch.tensor([nuc_d[x] for x in seq])
    vec=np.array([nuc_d[x] for x in seq]).flatten()
        
    return vec

In [12]:
class MultiTaskDataset(Dataset):
    def __init__(self,df):
        self.loci = list(df['locus_tag'])
        self.seqs = list(df['upstream_region'])
        self.seq_len = len(self.seqs[0])
    
        self.nocu_labels = list(df['NoCu'])
        self.lowcu_labels = list(df['lowCu'])
        self.medcu_labels = list(df['medCu'])
        self.highcu_labels = list(df['highCu'])
        
    def __len__(self): return len(self.loci)
    
    def __getitem__(self,idx):
        seq = torch.tensor(one_hot_encode(self.seqs[idx]))
        
        nocu = torch.tensor(float(self.nocu_labels[idx]))
        lowcu = torch.tensor(float(self.lowcu_labels[idx]))
        medcu = torch.tensor(float(self.medcu_labels[idx]))
        highcu = torch.tensor(float(self.highcu_labels[idx]))
        
        return seq, (nocu, lowcu, medcu, highcu)
        # ??? ^ how is this handled? Does the data loader know what to do with
        # a list of labels...?
        

In [13]:
def quick_load_and_split_input(df, split_frac=0.8, verbose=False):
    '''
    Given a df, randomly split between
    train and test. Not a formal train/test split, just a quick n dirty version.
    
    '''

    # train test split
    idxs = list(range(df.shape[0]))
    random.shuffle(idxs)

    split = int(len(idxs)*split_frac)
    train_idxs = idxs[:split]
    test_idxs = idxs[split:]
    
    # split df and convert to tensors
    train_df = df[df.index.isin(train_idxs)]
    test_df = df[df.index.isin(test_idxs)]
        
    return train_df, test_df


def build_dataloaders(df,batch_size=32):
    # train test split
    train_df, test_df = quick_load_and_split_input(df)
    
    # put into dataloader object
    train_ds = MultiTaskDataset(train_df)
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    
    test_ds = MultiTaskDataset(test_df)
    test_dl = DataLoader(test_ds, batch_size=batch_size * 2)
    
    return train_dl, test_dl,train_df,test_df

In [14]:
# did this actually work???
train_dl, test_dl,train_df, test_df = build_dataloaders(XYdf)

In [15]:
train_dl

In [16]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    print('*******')

tensor([[0., 0., 1.,  ..., 1., 0., 0.],
        [0., 1., 0.,  ..., 1., 0., 0.],
        [0., 0., 1.,  ..., 1., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 1.],
        [0., 0., 1.,  ..., 0., 1., 0.],
        [0., 1., 0.,  ..., 0., 1., 0.]], dtype=torch.float64)
[tensor([ 57.4526,  15.8513,  25.2045,  57.2232,  95.0641,  20.9470, 157.2000,
        565.0566,  89.2762,  58.7207, 886.0788,   0.0000, 206.4296,  53.3444,
         31.4181,  15.9749,   6.6779,   3.8572, 275.9895,  30.2502,  20.6546,
          8.9382,  48.0568,  25.6551,  11.5600,  15.0710,  94.1441, 133.8691,
         64.3297, 116.5948,  14.9377,  80.9924]), tensor([ 57.0908,  12.4401,  20.5634,  57.3638,  90.0727,  19.1010, 126.9318,
        586.9841,  67.6189,  52.8856, 994.4968,   0.0000, 150.3905,  54.9581,
         30.2176,  15.8771,   6.2018,   3.7174, 238.7724,  30.5601,  19.4110,
          7.3877,  47.3123,  24.2728,   9.3808,  15.0326,  83.3939, 110.1302,
         50.4473, 144.7868,  13.8450,  63.1770]), 

tensor([[0., 1., 0.,  ..., 0., 0., 1.],
        [1., 0., 0.,  ..., 1., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 1.],
        ...,
        [0., 1., 0.,  ..., 1., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 1.],
        [1., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)
[tensor([  64.4531,   32.6824,   89.5751,   25.2323,   56.0682,   59.9594,
          30.1208,   51.8238,   26.9085,   34.1492,   14.6181,   54.1326,
          62.8292,   38.5142,   27.9057,  140.5682, 1281.2480,  106.3304,
          30.4411, 1646.5765,  178.8220,   49.9468,  146.3354,  133.8345,
          77.2065,   12.4251,  118.8393,  603.4210,   53.3132,   66.7904,
          51.9758,   74.0779]), tensor([  62.1313,   36.9827,   82.8692,   23.0107,   53.2902,   53.3887,
          36.4628,   44.8918,   26.6823,   31.7016,   14.4856,   48.3111,
          67.3286,   37.6580,   23.8296,  129.4805, 1294.1810,  115.3336,
          26.4534, 1039.6091,  160.3840,   45.3788,  147.7827,  157.6620,
          71.5275,   10.93

[tensor([  3.2572,  17.6607, 151.4836,  95.7364,  38.8680, 157.8991,  14.8089,
          0.0000,  94.1744,   3.1564,  89.7540,  21.3440,   8.4027,   0.0000,
         23.7760,   8.6159,  41.8675, 319.8055,  36.1004,  40.9664,  26.2311,
        430.4978, 114.9411,  21.0129,   0.0000,  29.5927,  60.0767,  62.5589,
         26.2852, 138.9278,  63.7577, 135.1808]), tensor([  2.9559,  17.3504, 151.5593,  91.5878,  34.5746, 155.2073,  15.7662,
          0.0000,  88.2011,   2.9540, 105.5980,  17.7547,   9.5096,   0.0000,
         23.3294,   9.2897,  42.2034, 287.7898,  31.8812,  52.1509,  29.0843,
        509.9711, 127.3358,  21.7592,   0.0000,  28.6576,  58.0124,  49.0111,
         21.0329,  97.3232,  59.6665, 141.3152]), tensor([  2.5713,  15.7243, 138.0378,  88.7348,  28.4701, 138.1975,  14.2322,
          0.0000,  91.6558,   3.0243, 112.6045,  16.7883,   8.8817,   0.0000,
         19.5151,   9.6584,  37.0015, 282.2116,  27.2799,  56.1730,  38.2692,
        528.9481, 148.3075,  18.0215,   0

tensor([[0., 0., 1.,  ..., 1., 0., 0.],
        [0., 0., 1.,  ..., 0., 1., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 1., 0.,  ..., 0., 0., 1.],
        [0., 1., 0.,  ..., 1., 0., 0.],
        [0., 0., 1.,  ..., 1., 0., 0.]], dtype=torch.float64)
[tensor([   4.1831,   92.1448,   15.5674,  128.1308,   37.6818,  132.2145,
           9.1057, 3034.4563,  481.6253,  240.7278,  112.2584,  231.4232,
          17.0662,   12.9056,   26.8021,  115.6876,   58.6155,  211.9420,
        2892.0647,   42.2188,  100.8872,   24.8384,   61.4486, 1212.4197,
          90.0458,   21.4523,   52.1914,  432.1953,   40.1351,    6.6155,
          27.5036,   64.3018]), tensor([   4.2611,   87.1331,   12.9371,  120.8802,   35.2343,  120.4923,
           9.9639, 3260.9917,  468.9299,  231.6390,   99.5569,  165.4391,
          12.7714,   11.3402,   25.3764,  118.9057,   53.5123,  246.4681,
        3331.9507,   37.6329,   85.6990,   29.1140,   54.4734, 1424.6825,
          81.3513,   17.58

tensor([[0., 1., 0.,  ..., 0., 0., 1.],
        [0., 0., 1.,  ..., 1., 0., 0.],
        [1., 0., 0.,  ..., 1., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 1., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]], dtype=torch.float64)
[tensor([3.4238e+02, 5.5804e+01, 3.7398e+01, 1.6983e+02, 3.5241e+01, 1.5995e+02,
        3.3091e+02, 3.3115e+01, 2.0946e+01, 2.9697e+01, 8.1910e+01, 1.1621e+02,
        5.1274e+01, 1.7696e+01, 1.7486e+01, 1.5473e+01, 3.5044e+01, 8.0592e+01,
        7.4800e+01, 1.9134e+01, 2.4601e+02, 5.0347e+01, 1.1167e+02, 2.9313e+01,
        2.8582e+01, 5.3721e+03, 1.0162e+02, 3.7699e+01, 4.7836e+01, 1.3139e+02,
        3.7488e+00, 6.8149e+01]), tensor([3.5102e+02, 5.5576e+01, 3.2368e+01, 1.6157e+02, 3.3062e+01, 1.4590e+02,
        3.3726e+02, 2.8530e+01, 1.8186e+01, 2.9630e+01, 8.1447e+01, 1.0659e+02,
        5.2919e+01, 1.7357e+01, 1.5400e+01, 1.2049e+01, 3.1464e+01, 8.6491e+01,
        7.3177e+01, 1.6173e+01, 2.4860e+02, 4.4929

[tensor([1.6896e+02, 5.9703e+01, 5.7703e+01, 9.6886e+01, 3.6896e+01, 1.6355e+01,
        7.1880e+00, 3.6380e+01, 0.0000e+00, 1.6041e+01, 8.0718e+01, 2.1405e+00,
        1.2104e+02, 0.0000e+00, 9.1076e+01, 2.0349e+01, 6.3565e+01, 1.6888e+02,
        3.7753e+01, 1.3329e+01, 9.4498e+00, 3.2389e+01, 3.2138e+02, 4.1536e+01,
        2.2068e+03, 1.3699e+01, 2.4582e+02, 1.6240e+01, 8.3169e+00, 1.1748e+02,
        4.0098e+01, 1.3213e+01]), tensor([1.6046e+02, 5.5210e+01, 3.8388e+01, 8.8079e+01, 3.4504e+01, 1.4464e+01,
        6.9382e+00, 3.6350e+01, 0.0000e+00, 1.6112e+01, 7.7167e+01, 2.1220e+00,
        1.1292e+02, 0.0000e+00, 1.0869e+02, 1.6787e+01, 5.2128e+01, 1.6531e+02,
        3.4444e+01, 1.2753e+01, 8.5148e+00, 3.0237e+01, 2.9233e+02, 3.4733e+01,
        2.6875e+03, 1.1598e+01, 2.5372e+02, 1.7503e+01, 9.9307e+00, 1.1206e+02,
        4.0245e+01, 9.6233e+00]), tensor([1.3436e+02, 5.4461e+01, 2.8850e+01, 7.9548e+01, 3.2593e+01, 1.2042e+01,
        5.9246e+00, 3.7218e+01, 0.0000e+00, 1.4357e

tensor([[1., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 1.],
        [0., 0., 1.,  ..., 0., 0., 1.],
        [1., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)
[tensor([ 8.2193, 86.4518, 76.7569, 47.9235, 14.0302, 63.3489, 38.3847, 25.2129,
        50.3810,  5.2269]), tensor([ 7.5014, 84.0678, 73.5852, 45.0933, 16.3062, 62.0295, 31.3096, 24.9907,
        41.6491,  5.0146]), tensor([ 6.9560, 89.8796, 77.6459, 52.3342, 23.4459, 62.2770, 26.3080, 13.6674,
        37.6137,  6.1552]), tensor([  7.9818, 104.7552,  74.9064,  59.1036,  19.6287,  68.4303,  33.9764,
          5.5580,  36.7909,   6.6637])]
*******


In [ ]:
# ^^ yb seems ot be a list of tensors, each the length of a batch?

In [21]:
class DNA_Linear_Multi(nn.Module):
    def __init__(self,seq_len,h1_size,h2_size):
        super().__init__()
        
        # some arbitrary arch of a few linear layers
        self.lin_share = nn.Sequential(
            nn.Linear(4*seq_len, h1_size),
            nn.ReLU(inplace=True),
            nn.Linear(h1_size, h2_size),
            nn.ReLU(inplace=True)
        )
        
        # define the multi task objectectives?
        self.nocu_obj   = nn.Linear(h2_size,1)
        self.lowcu_obj  = nn.Linear(h2_size,1)
        self.medcu_obj  = nn.Linear(h2_size,1)
        self.highcu_obj = nn.Linear(h2_size,1)
        
        

    def forward(self, xb):
        # Linear wraps up the weights/bias dot product operations
        out = self.lin_share(xb)
        out_nocu = self.nocu_obj(out)
        out_lowcu = self.lowcu_obj(out)
        out_medcu = self.medcu_obj(out)
        out_highcu = self.highcu_obj(out)
        
        return [out_nocu, out_lowcu, out_medcu, out_highcu]
        # ^^ where does this go and what does it do?
    


In [68]:
h1 = 400
h2 = 200
multi_lin_model = DNA_Linear_Multi(100, h1,h2)

In [69]:
loss_func = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(multi_lin_model.parameters(), lr=0.0001) 

In [70]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    '''
    Apply loss function to a batch of inputs. If no optimizer
    is provided, skip the back prop step.
    '''
    #print("in Loss Batch...")

    #loss = loss_func(model(xb.float()), yb.float())
    # yb is a list of tensors... what should it be?
    # how to adapt loss for multi-task?
    #print("yb[0]:", yb[0])
    
    # try looping through ybs?
    res = model(xb.float()) # get predictions? returns list
    
    loss_nocu = loss_func(res[0], yb[0].float())
    loss_lowcu = loss_func(res[1], yb[1].float())
    loss_medcu = loss_func(res[2], yb[2].float())
    loss_highcu = loss_func(res[3], yb[3].float())
    
    loss = sum([loss_nocu, loss_lowcu, loss_medcu, loss_highcu])
    
    print("what is 'loss'?:")
    print(loss_nocu)
    print(loss_lowcu)
    print(loss_medcu)
    print(loss_highcu)
    print("*****")

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    #print("lb returning:",loss.item(), len(xb))
    return loss.item(), len(xb)


def fit(epochs, model, loss_func, opt, train_dl, test_dl):
    '''
    Fit the model params to the training data, eval on unseen data.
    Loop for a number of epochs and keep train of train and val losses 
    along the way
    '''
    # keep track of losses
    train_losses = []    
    val_losses = []
    
    # loops through epochs
    for epoch in range(epochs):
        print('Epoch:',epoch)
        #print("TRAIN")
        model.train()
        ts = []
        ns = []
        # collect train loss; provide opt so backpropo happens
        for xb, yb in train_dl:
            t, n = loss_batch(model, loss_func, xb, yb, opt)
            ts.append(t)
            ns.append(n)
        train_loss = np.sum(np.multiply(ts, ns)) / np.sum(ns)
        train_losses.append(train_loss)
        
        #print("EVAL")
        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                # loop through test batches
                # returns loss calc for test set batch size
                # unzips into two lists
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in test_dl]
                # Note: no opt provided, backprop won't happen
            )
        # Gets average MSE loss across all batches (may be of diff sizes, hence the multiply)
        #print("losses", losses)
        #print("nums", nums)
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss)
        val_losses.append(val_loss)

    return train_losses, val_losses

In [71]:
epochs = 20

train_losses, val_losses = fit(epochs, multi_lin_model, loss_func, optimizer, train_dl, test_dl)

Epoch: 0
what is 'loss'?:
tensor(64548.2930, grad_fn=<MseLossBackward>)
tensor(98027.2031, grad_fn=<MseLossBackward>)
tensor(110896.1562, grad_fn=<MseLossBackward>)
tensor(99123.4688, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(243965.7812, grad_fn=<MseLossBackward>)
tensor(298228.7188, grad_fn=<MseLossBackward>)
tensor(315704.4375, grad_fn=<MseLossBackward>)
tensor(322020.7500, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(66252.6562, grad_fn=<MseLossBackward>)
tensor(75454.8906, grad_fn=<MseLossBackward>)
tensor(97184.6875, grad_fn=<MseLossBackward>)
tensor(108635.1406, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(97658.7266, grad_fn=<MseLossBackward>)
tensor(138933.5625, grad_fn=<MseLossBackward>)
tensor(146969.5156, grad_fn=<MseLossBackward>)
tensor(180067.1094, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(17498.4922, grad_fn=<MseLossBackward>)
tensor(16036.3896, grad_fn=<MseLossBackward>)
tensor(13601.4814, grad_fn=<MseLossBackward>)

tensor(14105872., grad_fn=<MseLossBackward>)
tensor(23151940., grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(63618.5938, grad_fn=<MseLossBackward>)
tensor(98763.7812, grad_fn=<MseLossBackward>)
tensor(115884.1562, grad_fn=<MseLossBackward>)
tensor(109312.6562, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(976066.6875, grad_fn=<MseLossBackward>)
tensor(879121.4375, grad_fn=<MseLossBackward>)
tensor(797854.5625, grad_fn=<MseLossBackward>)
tensor(1308351.1250, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(3808.9521, grad_fn=<MseLossBackward>)
tensor(3346.6875, grad_fn=<MseLossBackward>)
tensor(3208.8340, grad_fn=<MseLossBackward>)
tensor(4243.4287, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(9504.0283, grad_fn=<MseLossBackward>)
tensor(9730.0674, grad_fn=<MseLossBackward>)
tensor(9911.0781, grad_fn=<MseLossBackward>)
tensor(11577.5840, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(245625.2656, grad_fn=<MseLossBackward>)
tensor(26620

what is 'loss'?:
tensor(111599.5625, grad_fn=<MseLossBackward>)
tensor(126275.4375, grad_fn=<MseLossBackward>)
tensor(147456.5469, grad_fn=<MseLossBackward>)
tensor(199218.7656, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(45836.8359, grad_fn=<MseLossBackward>)
tensor(61937.8750, grad_fn=<MseLossBackward>)
tensor(72932.2109, grad_fn=<MseLossBackward>)
tensor(89716.7891, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(81890.0625, grad_fn=<MseLossBackward>)
tensor(119229.0312, grad_fn=<MseLossBackward>)
tensor(172414.5000, grad_fn=<MseLossBackward>)
tensor(127658.5547, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(246284.6719, grad_fn=<MseLossBackward>)
tensor(356817.8750, grad_fn=<MseLossBackward>)
tensor(728525.3125, grad_fn=<MseLossBackward>)
tensor(484301.6250, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(112707.7734, grad_fn=<MseLossBackward>)
tensor(125595.5000, grad_fn=<MseLossBackward>)
tensor(228053., grad_fn=<MseLossBackward>)
tensor(

what is 'loss'?:
tensor(4980660.5000, grad_fn=<MseLossBackward>)
tensor(5151231.5000, grad_fn=<MseLossBackward>)
tensor(3788013., grad_fn=<MseLossBackward>)
tensor(3574328., grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(367055.5312, grad_fn=<MseLossBackward>)
tensor(454764.4062, grad_fn=<MseLossBackward>)
tensor(452884.2188, grad_fn=<MseLossBackward>)
tensor(393365.1875, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(80922.2578, grad_fn=<MseLossBackward>)
tensor(120676.6484, grad_fn=<MseLossBackward>)
tensor(125310.7031, grad_fn=<MseLossBackward>)
tensor(153692.8750, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(19809.1602, grad_fn=<MseLossBackward>)
tensor(15863.1152, grad_fn=<MseLossBackward>)
tensor(11860.7031, grad_fn=<MseLossBackward>)
tensor(13539.5195, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(128060.6484, grad_fn=<MseLossBackward>)
tensor(58751.1016, grad_fn=<MseLossBackward>)
tensor(25271.2773, grad_fn=<MseLossBackward>)
tensor(34

what is 'loss'?:
tensor(59531.9297, grad_fn=<MseLossBackward>)
tensor(107629.7266, grad_fn=<MseLossBackward>)
tensor(173262.4688, grad_fn=<MseLossBackward>)
tensor(123057.5312, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(17072.4551, grad_fn=<MseLossBackward>)
tensor(14571.6191, grad_fn=<MseLossBackward>)
tensor(13051.3311, grad_fn=<MseLossBackward>)
tensor(20436.0781, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(172003.1250, grad_fn=<MseLossBackward>)
tensor(247970.5000, grad_fn=<MseLossBackward>)
tensor(472694.3125, grad_fn=<MseLossBackward>)
tensor(377295.7812, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(148544.9531, grad_fn=<MseLossBackward>)
tensor(192437.6250, grad_fn=<MseLossBackward>)
tensor(202502.2969, grad_fn=<MseLossBackward>)
tensor(197904.3594, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(57156.9219, grad_fn=<MseLossBackward>)
tensor(56867.1836, grad_fn=<MseLossBackward>)
tensor(67053.0703, grad_fn=<MseLossBackward>)
tensor

...

what is 'loss'?:
tensor(4509.7622, grad_fn=<MseLossBackward>)
tensor(4208.9370, grad_fn=<MseLossBackward>)
tensor(3925.2744, grad_fn=<MseLossBackward>)
tensor(5221.0439, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(8454.8564, grad_fn=<MseLossBackward>)
tensor(7434.7310, grad_fn=<MseLossBackward>)
tensor(6816.2744, grad_fn=<MseLossBackward>)
tensor(10215.0371, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(4754853., grad_fn=<MseLossBackward>)
tensor(5015517.5000, grad_fn=<MseLossBackward>)
tensor(3715116.5000, grad_fn=<MseLossBackward>)
tensor(3507356.5000, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(10339.2393, grad_fn=<MseLossBackward>)
tensor(9361.3340, grad_fn=<MseLossBackward>)
tensor(11463.3643, grad_fn=<MseLossBackward>)
tensor(12952.6504, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(61392.4766, grad_fn=<MseLossBackward>)
tensor(64273.0508, grad_fn=<MseLossBackward>)
tensor(110325.4453, grad_fn=<MseLossBackward>)
tensor(113791.0391, 

what is 'loss'?:
tensor(21070.3750, grad_fn=<MseLossBackward>)
tensor(22259.9824, grad_fn=<MseLossBackward>)
tensor(33772.1211, grad_fn=<MseLossBackward>)
tensor(35675.9922, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(4480750.5000, grad_fn=<MseLossBackward>)
tensor(2329610.5000, grad_fn=<MseLossBackward>)
tensor(167855.7500, grad_fn=<MseLossBackward>)
tensor(109069.1953, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(92740.2969, grad_fn=<MseLossBackward>)
tensor(111469., grad_fn=<MseLossBackward>)
tensor(224493.4531, grad_fn=<MseLossBackward>)
tensor(191522.3125, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(42613.3945, grad_fn=<MseLossBackward>)
tensor(36541.0352, grad_fn=<MseLossBackward>)
tensor(37121.4102, grad_fn=<MseLossBackward>)
tensor(59873.6094, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(139865.4375, grad_fn=<MseLossBackward>)
tensor(181582.3438, grad_fn=<MseLossBackward>)
tensor(198714.7188, grad_fn=<MseLossBackward>)
tensor(18

what is 'loss'?:
tensor(455476.8438, grad_fn=<MseLossBackward>)
tensor(398629.9688, grad_fn=<MseLossBackward>)
tensor(457166.7812, grad_fn=<MseLossBackward>)
tensor(605607.9375, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(16489.7305, grad_fn=<MseLossBackward>)
tensor(17656.7344, grad_fn=<MseLossBackward>)
tensor(22604.7695, grad_fn=<MseLossBackward>)
tensor(20599.6367, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(95638.9297, grad_fn=<MseLossBackward>)
tensor(33347.1836, grad_fn=<MseLossBackward>)
tensor(3401.1992, grad_fn=<MseLossBackward>)
tensor(3941.8459, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(171812.8594, grad_fn=<MseLossBackward>)
tensor(249418.4844, grad_fn=<MseLossBackward>)
tensor(482088.7188, grad_fn=<MseLossBackward>)
tensor(390597., grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(72117.6406, grad_fn=<MseLossBackward>)
tensor(68314.0078, grad_fn=<MseLossBackward>)
tensor(135855.3906, grad_fn=<MseLossBackward>)
tensor(193344.

what is 'loss'?:
tensor(11123.4775, grad_fn=<MseLossBackward>)
tensor(9658.5977, grad_fn=<MseLossBackward>)
tensor(8822.7656, grad_fn=<MseLossBackward>)
tensor(14341.3105, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(101448.3672, grad_fn=<MseLossBackward>)
tensor(157932.8594, grad_fn=<MseLossBackward>)
tensor(176782.2656, grad_fn=<MseLossBackward>)
tensor(198656.5469, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(60981.7383, grad_fn=<MseLossBackward>)
tensor(67314.2656, grad_fn=<MseLossBackward>)
tensor(78624.5391, grad_fn=<MseLossBackward>)
tensor(90101.9453, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(1908619.2500, grad_fn=<MseLossBackward>)
tensor(2889851.2500, grad_fn=<MseLossBackward>)
tensor(4200878., grad_fn=<MseLossBackward>)
tensor(8809002., grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(346537.2188, grad_fn=<MseLossBackward>)
tensor(317920.5312, grad_fn=<MseLossBackward>)
tensor(291445.5312, grad_fn=<MseLossBackward>)
tensor(21570

what is 'loss'?:
tensor(165266.1406, grad_fn=<MseLossBackward>)
tensor(183334.1250, grad_fn=<MseLossBackward>)
tensor(200807.7500, grad_fn=<MseLossBackward>)
tensor(125216.1172, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(8720.2227, grad_fn=<MseLossBackward>)
tensor(8970.5801, grad_fn=<MseLossBackward>)
tensor(12771.5293, grad_fn=<MseLossBackward>)
tensor(10397.3115, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(29251.4922, grad_fn=<MseLossBackward>)
tensor(20582.0957, grad_fn=<MseLossBackward>)
tensor(19309.0391, grad_fn=<MseLossBackward>)
tensor(23161.0684, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(18548.6719, grad_fn=<MseLossBackward>)
tensor(19372.2969, grad_fn=<MseLossBackward>)
tensor(17966.9062, grad_fn=<MseLossBackward>)
tensor(20941.2793, grad_fn=<MseLossBackward>)
*****
what is 'loss'?:
tensor(295712.5000, grad_fn=<MseLossBackward>)
tensor(137629.3438, grad_fn=<MseLossBackward>)
tensor(70923.8125, grad_fn=<MseLossBackward>)
tensor(198485

### Ok this seems to be running. The loss is exploding upward. Not yet sure if that's because it's a terrible model or if I've done something incorrectly. TBD

In [30]:
for xb, yb in train_dl:
    pass

In [53]:
type(xb[0])

torch.Tensor

In [57]:
type(multi_lin_model(xb.float()))

list

In [11]:
def format_XY_dfs(tpm_df, meta_df):
    # melt tpm df so every sample is in a row
    tpm_melt = tpm_df[['locus_tag']+samples].melt(id_vars=['locus_tag'],var_name='sample',value_name='tpm')
    tpm_melt['condition'] = tpm_melt['sample'].apply(lambda x: sample2condition[x])

    # also add in upstream seq
    tpm_melt['upstream_region'] = tpm_melt['locus_tag'].apply(lambda x: loc2seq[x])
    
    # get coded metadata conditions from meta_df
    samp2cond_df = meta_df[['sample']+COND_COLS]

    # merge back onto the tpm df
    df = tpm_melt.merge(samp2cond_df,on='sample',how='left')
    
    # reformat full df
    full_df = df[['locus_tag','upstream_region', 'sample','condition']+COND_COLS+['tpm']]
    
    # separate out just X (features) and Y (labels)
    X = full_df[['locus_tag','upstream_region']+COND_COLS]
    Y = full_df['tpm']
    
    return full_df,X,Y

In [12]:
full_df, X, Y = format_XY_dfs(tpm_df,meta_df)
full_df.head()

,locus_tag,upstream_region,sample,condition,carbon_source,oxygen_level,nitrate_level,copper_level,lanthanum_level,growth_rate,growth_mode,tpm
0,EQU24_RS00005,CGCCGGTTTATGTCAATTATGCCGGCACTGATTTGATTGCTGTATA...,5GB1_ferm_Ack_QC_tpm,lowO2_slow_growth,2,0,0,3,0,0,0,2.933003
1,EQU24_RS00010,AACGCCGGTTTTACAGTTCATAAGCTATTGATAAATAAAATAAAAA...,5GB1_ferm_Ack_QC_tpm,lowO2_slow_growth,2,0,0,3,0,0,0,1.607784
2,EQU24_RS00015,ATCGCAGTCATTATTAAATGTGGAAGCAACAAAAAAACGAGCTTGT...,5GB1_ferm_Ack_QC_tpm,lowO2_slow_growth,2,0,0,3,0,0,0,1.415515
3,EQU24_RS00020,AACTTAATAACTATAAAATGTTCCACGTGGAACATGGTGAAATTAA...,5GB1_ferm_Ack_QC_tpm,lowO2_slow_growth,2,0,0,3,0,0,0,3.200081
4,EQU24_RS00025,CTTTGCCGAACACCCCGCACCTCCACGCGTCAACAACGAAATTTGA...,5GB1_ferm_Ack_QC_tpm,lowO2_slow_growth,2,0,0,3,0,0,0,1.522728


In [13]:
tpm_df.head()

,locus_tag,product,type,gene_symbol,locus,start_coord,end_coord,note,translation,gene_len,...,5GB1_pA9_red_tpm,5GB1_pA9_yellow_tpm,5GB1C-5G-La-BR1_tpm,5GB1C-5G-La-BR2_tpm,5GB1C-5G-N-BR1_tpm,5GB1C-5G-N-BR2_tpm,5GB1C-JG15-La-BR1_tpm,5GB1C-JG15-La-BR2_tpm,5GB1C-JG15-N-BR1_tpm,5GB1C-JG15-N-BR2_tpm
0,EQU24_RS00005,chromosomal replication initiator protein DnaA,CDS,dnaA,NZ_CP035467.1,0,1317,Derived by automated computational analysis us...,MSALWNNCLAKLENEISSSEFSTWIRPLQAIETDGQIKLLAPNRFV...,1318,...,38.557373,38.810668,37.444214,40.246006,40.100118,33.432274,39.880174,38.355431,30.247582,41.248441
1,EQU24_RS00010,DNA polymerase III subunit beta,CDS,,NZ_CP035467.1,1502,2603,Derived by automated computational analysis us...,MKYIINREQLLVPLQQIVSVIEKRQTMPILSNVLMVFRENTLVMTG...,1102,...,52.552767,52.461746,42.676553,49.210083,46.798476,48.142385,45.465136,46.498139,37.152951,52.902410
2,EQU24_RS00015,DNA replication/repair protein RecF,CDS,recF,NZ_CP035467.1,3060,4140,Derived by automated computational analysis us...,MSLQKLDIFNVRNIRQASLQPSPGLNLIYGANASGKSSVLEAIFIL...,1081,...,31.350991,34.914128,21.479309,24.204682,22.171104,22.006566,22.658157,22.753325,19.407103,29.834124
3,EQU24_RS00020,DNA topoisomerase (ATP-hydrolyzing) subunit B,CDS,gyrB,NZ_CP035467.1,4185,6600,Derived by automated computational analysis us...,MSENIKQYDSTNIQVLKGLDAVRKRPGMYIGDTDDGTGLHHMVFEV...,2416,...,74.848501,80.850761,54.959319,64.911376,59.653059,64.648318,69.119079,65.643179,57.590223,68.306759
4,EQU24_RS00025,hypothetical protein,CDS,,NZ_CP035467.1,6825,7062,Derived by automated computational analysis us...,VKTTKYFLTTRMRPDREIIKDEWIQYVVRFPENEHIQFDGRIRRWA...,238,...,50.324948,49.349547,34.539657,36.521074,37.789611,39.358066,38.992158,35.870964,41.462392,40.227192


In [15]:
full_df.head()

,locus_tag,upstream_region,sample,condition,carbon_source,oxygen_level,nitrate_level,copper_level,lanthanum_level,growth_rate,growth_mode,tpm
0,EQU24_RS00005,CGCCGGTTTATGTCAATTATGCCGGCACTGATTTGATTGCTGTATA...,5GB1_ferm_Ack_QC_tpm,lowO2_slow_growth,2,0,0,3,0,0,0,2.933003
1,EQU24_RS00010,AACGCCGGTTTTACAGTTCATAAGCTATTGATAAATAAAATAAAAA...,5GB1_ferm_Ack_QC_tpm,lowO2_slow_growth,2,0,0,3,0,0,0,1.607784
2,EQU24_RS00015,ATCGCAGTCATTATTAAATGTGGAAGCAACAAAAAAACGAGCTTGT...,5GB1_ferm_Ack_QC_tpm,lowO2_slow_growth,2,0,0,3,0,0,0,1.415515
3,EQU24_RS00020,AACTTAATAACTATAAAATGTTCCACGTGGAACATGGTGAAATTAA...,5GB1_ferm_Ack_QC_tpm,lowO2_slow_growth,2,0,0,3,0,0,0,3.200081
4,EQU24_RS00025,CTTTGCCGAACACCCCGCACCTCCACGCGTCAACAACGAAATTTGA...,5GB1_ferm_Ack_QC_tpm,lowO2_slow_growth,2,0,0,3,0,0,0,1.522728


In [18]:
len(X.upstream_region.values[0])

100

In [20]:
X

,locus_tag,upstream_region,carbon_source,oxygen_level,nitrate_level,copper_level,lanthanum_level,growth_rate,growth_mode
0,EQU24_RS00005,CGCCGGTTTATGTCAATTATGCCGGCACTGATTTGATTGCTGTATA...,2,0,0,3,0,0,0
1,EQU24_RS00010,AACGCCGGTTTTACAGTTCATAAGCTATTGATAAATAAAATAAAAA...,2,0,0,3,0,0,0
2,EQU24_RS00015,ATCGCAGTCATTATTAAATGTGGAAGCAACAAAAAAACGAGCTTGT...,2,0,0,3,0,0,0
3,EQU24_RS00020,AACTTAATAACTATAAAATGTTCCACGTGGAACATGGTGAAATTAA...,2,0,0,3,0,0,0
4,EQU24_RS00025,CTTTGCCGAACACCCCGCACCTCCACGCGTCAACAACGAAATTTGA...,2,0,0,3,0,0,0
...,...,...,...,...,...,...,...,...,...
412869,EQU24_RS22135,CCCGGCCGGTTTGGTCTTGTACTGGGTGGTCAACAATACGCTGTCG...,2,1,0,3,0,2,1
412870,EQU24_RS22140,GCCGCCCAGGGCACCTATCTTACAGTCCGAAGAGTATTAAAGTGTC...,2,1,0,3,0,2,1
412871,EQU24_RS22145,AATATTGATGTTGTTGTTATGGCCCGAAAAGATGCACTCAATGCAT...,2,1,0,3,0,2,1
412872,EQU24_RS22150,AAGAACTCACGGCTTTCGTGCCAGAATGGCGACCAAAGGCGGCCGT...,2,1,0,3,0,2,1


## Linear model arch

In [ ]:
class DNA_Linear(nn.Module):
    def __init__(self, seq_len,h1_size,num_heads):
        super().__init__()
        self.lin = nn.Linear(4*seq_len, h1_size)

    def forward(self, xb):
        # Linear wraps up the weights/bias dot product operations
        return self.lin(xb)